<a href="https://colab.research.google.com/github/biaadeodato/PI-5/blob/main/Kamisado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from copy import deepcopy
import random
import math

white_labels = ["B1", "B2", "B3", "B4", "B5", "B6", "B7", "B8"]
black_labels = ["P1", "P2", "P3", "P4", "P5", "P6", "P7", "P8"]

board = np.array([
    ["laranja", "azul", "roxo", "rosa", "amarelo", "vermelho", "verde", "marrom"],
    ["vermelho", "laranja", "rosa", "verde", "azul", "amarelo", "marrom", "roxo"],
    ["verde", "rosa", "laranja", "vermelho", "roxo", "marrom", "amarelo", "azul"],
    ["rosa", "roxo", "azul", "laranja", "marrom", "verde", "vermelho", "amarelo"],
    ["amarelo", "vermelho", "verde", "marrom", "laranja", "azul", "roxo", "rosa"],
    ["azul", "amarelo", "marrom", "roxo", "vermelho", "laranja", "rosa", "verde"],
    ["roxo", "marrom", "amarelo", "azul", "verde", "rosa", "laranja", "vermelho"],
    ["marrom", "verde", "vermelho", "amarelo", "rosa", "roxo", "azul", "laranja"]
])

color_emojis = {
    "laranja": "🟧", "azul": "🟦", "roxo": "🟪", "rosa": "🩷",
    "amarelo": "🟨", "vermelho": "🟥", "verde": "🟩", "marrom": "🟫"
}

def initial_state():
    return {
        "white": {label: (7, i) for i, label in enumerate(white_labels)},
        "black": {label: (0, i) for i, label in enumerate(black_labels)},
        "turn": "white",
        "next_color": None
    }

def print_game_state(state):
    pos_map = {pos: label for label, pos in {**state["white"], **state["black"]}.items()}

    print("\n     " + " ".join(f"{chr(65 + c)}".center(5) for c in range(8)))
    for row in range(8):
        line = f"{8 - row}  |"
        for col in range(8):
            color = board[row, col]
            label = pos_map.get((row, col), '  ')
            line += f"{color_emojis[color]}{label}".ljust(5)
        line += f"|  {8 - row}"
        print(line)
    print("     " + " ".join(f"{chr(65 + c)}".center(5) for c in range(8)))

# Lógica
def successors(state):
    current_player = state["turn"]
    opponent = "black" if current_player == "white" else "white"
    moves = []

    # Filtra peças pela cor obrigatória
    valid_pieces = [
        label for label in state[current_player]
        if state["next_color"] is None or board[state[current_player][label][0]][state[current_player][label][1]] == state["next_color"]
    ]

    for label in valid_pieces:
        row, col = state[current_player][label]
        directions = [(-1, 0), (-1, -1), (-1, 1)] if current_player == "white" else [(1, 0), (1, -1), (1, 1)]

        for dr, dc in directions:
            step = 1
            while True:
                new_row = row + (dr * step)
                new_col = col + (dc * step)

                if not (0 <= new_row < 8 and 0 <= new_col < 8):
                    break

                if any((new_row, new_col) == pos for pos in {**state["white"], **state["black"]}.values()):
                    break

                new_state = deepcopy(state)
                new_state[current_player][label] = (new_row, new_col)
                new_state["next_color"] = board[new_row][new_col]
                new_state["turn"] = opponent
                moves.append(new_state)

                step += 1

    return moves

def game_over(state):
    for pos in state["white"].values():
        if pos[0] == 0:
            return "white"
    for pos in state["black"].values():
        if pos[0] == 7:
            return "black"
    return None

# IA
def advanced_heuristic(state):
    score = 0

    # 1. Fator Material (40%)
    material = (len(state["black"]) - len(state["white"])) * 200

    # 2. Progressão (30%)
    progression = sum((7 - row) * 25 for row, _ in state["black"].values())  # IA
    progression -= sum(row * 30 for row, _ in state["white"].values())       # Jogador

    # 3. Controle do Centro (20%)
    center_squares = [(3,3), (3,4), (4,3), (4,4)]
    center_control = sum(
        50 if (row, col) in state["black"].values() else
        -40 if (row, col) in state["white"].values() else 0
        for row, col in center_squares
    )

    # 4. Mobilidade (10%)
    mobility = len(successors(state)) * (10 if state["turn"] == "black" else -8)

    # 5. Bônus de Cor Vinculante
    color_bonus = 0
    if state["next_color"]:
        color_bonus = sum(
            35 for pos in state["black"].values()
            if board[pos[0]][pos[1]] == state["next_color"]
        )

    # Cálculo final
    score = (material * 0.4) + (progression * 0.3) + (center_control * 0.2) + (mobility * 0.1) + color_bonus
    return int(score)

def minimax(state, depth, alpha, beta, maximizing_player):
    if depth == 0 or game_over(state):
        return advanced_heuristic(state), state

    if maximizing_player:
        max_eval = -math.inf
        best_move = None
        for move in successors(state):
            eval, _ = minimax(move, depth-1, alpha, beta, False)
            if eval > max_eval:
                max_eval = eval
                best_move = move
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval, best_move
    else:
        min_eval = math.inf
        best_move = None
        for move in successors(state):
            eval, _ = minimax(move, depth-1, alpha, beta, True)
            if eval < min_eval:
                min_eval = eval
                best_move = move
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval, best_move

def computer_move(state):
    _, best_move = minimax(state, depth=3, alpha=-math.inf, beta=math.inf, maximizing_player=True)
    return best_move or random.choice(successors(state))

# Loop principal
def play_game():
    state = initial_state()

    while True:
        print_game_state(state)
        winner = game_over(state)
        if winner:
            print(f"\nFim de jogo! {'Você' if winner == 'white' else 'Computador'} venceu!")
            break

        if state["turn"] == "white":
            print("\nSua vez (Brancas):")
            if state["next_color"]:
                print(f"Você DEVE jogar uma peça na cor: {state['next_color']}")

            available_pieces = [
                label for label in state["white"]
                if state["next_color"] is None or board[state["white"][label][0]][state["white"][label][1]] == state["next_color"]
            ]

            print("Peças disponíveis:", ", ".join(available_pieces))

            piece = input("Selecione sua peça (ex: B1): ").strip().upper()
            while piece not in available_pieces:
                print("Peça inválida! Escolha entre:", ", ".join(available_pieces))
                piece = input("Selecione sua peça: ").strip().upper()

            valid_moves = []
            for move in successors(state):
                if move["white"][piece] != state["white"][piece]:
                    new_row, new_col = move["white"][piece]
                    col_char = chr(65 + new_col)
                    row_num = 8 - new_row
                    valid_moves.append((f"{col_char}{row_num}", move))

            print("\nMovimentos possíveis:")
            for i, (pos, _) in enumerate(valid_moves):
                print(f"{i}: Mover para {pos}")

            choice = int(input("Escolha o movimento: "))
            while choice < 0 or choice >= len(valid_moves):
                print("Número inválido! Escolha entre 0 e", len(valid_moves)-1)
                choice = int(input("Escolha o movimento: "))

            state = valid_moves[choice][1]
        else:
            print("\nVez do computador (Pretas)...")
            state = computer_move(state)

if __name__ == "__main__":
    play_game()


       A     B     C     D     E     F     G     H  
8  |🟧P1  🟦P2  🟪P3  🩷P4  🟨P5  🟥P6  🟩P7  🟫P8  |  8
7  |🟥    🟧    🩷    🟩    🟦    🟨    🟫    🟪    |  7
6  |🟩    🩷    🟧    🟥    🟪    🟫    🟨    🟦    |  6
5  |🩷    🟪    🟦    🟧    🟫    🟩    🟥    🟨    |  5
4  |🟨    🟥    🟩    🟫    🟧    🟦    🟪    🩷    |  4
3  |🟦    🟨    🟫    🟪    🟥    🟧    🩷    🟩    |  3
2  |🟪    🟫    🟨    🟦    🟩    🩷    🟧    🟥    |  2
1  |🟫B1  🟩B2  🟥B3  🟨B4  🩷B5  🟪B6  🟦B7  🟧B8  |  1
       A     B     C     D     E     F     G     H  

Sua vez (Brancas):
Peças disponíveis: B1, B2, B3, B4, B5, B6, B7, B8
